# Importing Data and Libraries

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
con = sqlite3.connect('data/im.db')
cur = con.cursor()

In [3]:
bmg = pd.read_csv('data/bom.movie_gross.csv')
mb = pd.read_csv('data/tn.movie_budgets.csv')
m = pd.read_csv('data/tmdb.movies.csv')

In [4]:
rv = pd.read_csv('data/rt.reviews.tsv', delimiter = '\t', encoding = 'latin') 
# needed to change the encoding as it was throwing an error message about it
# tsv file, so needed to add a delimiter of tab

In [5]:
mi = pd.read_csv('data/rt.movie_info.tsv', delimiter = '\t')

In [6]:
imdb_rated = pd.read_csv('data/IMDB_rated_only.csv')

# Exploring the im.db Data

In [7]:
%%bash

sqlite3 data/im.db
.tables

#checking the table names

directors      movie_akas     movie_ratings  principals   
known_for      movie_basics   persons        writers      


In [8]:
%%bash

sqlite3 data/im.db
.schema

#checking schema

CREATE TABLE IF NOT EXISTS "movie_basics" (
"movie_id" TEXT,
  "primary_title" TEXT,
  "original_title" TEXT,
  "start_year" INTEGER,
  "runtime_minutes" REAL,
  "genres" TEXT
);
CREATE TABLE IF NOT EXISTS "directors" (
"movie_id" TEXT,
  "person_id" TEXT
);
CREATE TABLE IF NOT EXISTS "known_for" (
"person_id" TEXT,
  "movie_id" TEXT
);
CREATE TABLE IF NOT EXISTS "movie_akas" (
"movie_id" TEXT,
  "ordering" INTEGER,
  "title" TEXT,
  "region" TEXT,
  "language" TEXT,
  "types" TEXT,
  "attributes" TEXT,
  "is_original_title" REAL
);
CREATE TABLE IF NOT EXISTS "movie_ratings" (
"movie_id" TEXT,
  "averagerating" REAL,
  "numvotes" INTEGER
);
CREATE TABLE IF NOT EXISTS "persons" (
"person_id" TEXT,
  "primary_name" TEXT,
  "birth_year" REAL,
  "death_year" REAL,
  "primary_profession" TEXT
);
CREATE TABLE IF NOT EXISTS "principals" (
"movie_id" TEXT,
  "ordering" INTEGER,
  "person_id" TEXT,
  "category" TEXT,
  "job" TEXT,
  "characters" TEXT
);
CREATE TABLE IF NOT EXISTS "writers" (
"m

In [9]:
query_result = pd.read_sql('''
    SELECT *
    FROM 
        movie_ratings AS mr 
        INNER JOIN movie_basics AS mb
            ON mb.movie_id = mr.movie_id
    
    ORDER BY "averagerating" DESC

    LIMIT 50
    
''', con)

query_result

DatabaseError: Execution failed on sql '
    SELECT *
    FROM 
        movie_ratings AS mr 
        INNER JOIN movie_basics AS mb
            ON mb.movie_id = mr.movie_id
    
    ORDER BY "averagerating" DESC

    LIMIT 50
    
': no such table: movie_ratings

In [ ]:
query_result = pd.read_sql('''
    SELECT DISTINCT *
    FROM directors, movie_akas, movie_ratings, principals,  
          known_for, movie_basics, persons, writers 
    LIMIT 5
''', con)

query_result

In [ ]:
con.close()

# Cleaning CSV files

In [ ]:
bmg = bmg.reset_index()

In [ ]:
bmg['foreign_gross'] = bmg['foreign_gross'].str.replace(',','')

In [ ]:
bmg['foreign_gross'] = bmg['foreign_gross'].astype(float)

In [ ]:
mb['domestic_gross'] = mb['domestic_gross'].str.replace('$','')
mb['domestic_gross'] = mb['domestic_gross'].str.replace(',','')
mb['domestic_gross'] = mb['domestic_gross'].astype(int)

In [ ]:
mb['worldwide_gross'] = mb['worldwide_gross'].str.replace('$','')

In [ ]:
mb['worldwide_gross'] = mb['worldwide_gross'].str.replace(',','')

In [ ]:
mb['worldwide_gross'] = mb['worldwide_gross'].astype(int)

In [ ]:
mb['foreign_gross'] = mb['worldwide_gross'] - mb['domestic_gross']

In [ ]:
mb['release_date'] = pd.to_datetime(mb['release_date'])

In [ ]:
mb['release_year'] = mb['release_date'].dt.year

In [ ]:
m['release_date'] = pd.to_datetime(m['release_date'])

In [ ]:
mb['release_year'] = mb['release_date'].dt.year

In [ ]:
m['release_year'] = m['release_date'].dt.year

In [ ]:
print(bmg['year'].dtype)

# Exploring the Movie Gross and Movie Budgets files

In [ ]:
bmg.info()
# looks like foreign_gross is a string when it should be an integer

In [ ]:
bmg.isna().sum()
#foreign gross has a ton of null values

In [ ]:
mb.notna().sum()

In [ ]:
mb.head()

# Combining Data: bmg, mb, m

In [ ]:
csvmerge =  m.merge(mb, how = 'left', 
                    left_on =['original_title', 'release_year'], 
                    right_on =['movie', 'release_year']).merge(bmg, how = 'left',
                                left_on =['original_title', 'release_year'], 
                                right_on =['title', 'year'])

In [ ]:
#print(m.count())
#print(mb.count())
#print(bmg.count())
csvmerge.count()
#checking to see if merge worked ok and how many rows merged well together

In [ ]:
csvmerge.sample(1)

In [ ]:
print(bmg.columns)
print(csvmerge.columns)
csvmerge.info()

In [ ]:
allcsvs_merged = imdb_rated.merge(csvmerge, how = 'left',
                 left_on = ['primary_title', 'start_year'], 
                 right_on = ['movie', 'release_year']).dropna().drop_duplicates(subset = ['movie','start_year'])

# Exploring Combined CSV tables

In [ ]:
allcsvs_merged.info()

In [ ]:
allcsvs_merged.tail(1)

In [ ]:
allcsvs_merged.shape

In [ ]:
allcsvs_merged.columns

In [ ]:
allcsvs_merged.sort_values(by='worldwide_gross')

In [ ]:
allcsvs_merged.sample(4)

In [ ]:
pd.set_option('display.max_rows', None)
allcsvs_merged.sort_values(by='primary_title')

In [ ]:
pd.set_option('display.max_rows', 50)

In [ ]:
allcsvs_merged.columns

# Cleaning the Merged Data

In [ ]:
cleancm = allcsvs_merged[['title_x', 'genres', 'original_language', 'foreign_gross_x', 'foreign_gross_y', 'worldwide_gross', 
                     'release_date_x', 'domestic_gross_x', 'domestic_gross_y', 
                     'production_budget', 'popularity', 'vote_average',
                     'vote_count', 'release_year', 'id_y', 'release_date_y',
                       'studio',
                      'averagerating', 'numvotes',
                       'runtime_minutes', ]]
cleancm.head()

In [ ]:
cleancm.info()

In [ ]:
cleancm.sort_values(by='foreign_gross_x', ascending=True).head(20)
#cleancm.sort_values(by='foreign_gross_y', ascending=True).head(20)

# compared foreign_gross_x values with foreign_gross_y values
# looks like there is some bad data in both
# Seems like there is less bad data in is foreign_gross_x
# need to drop two rows of bad data in it, and use foreign_gross_x for visualizations

In [ ]:
cleancm = cleancm.drop([67902, 37757])

In [ ]:
cleancm.sort_values(by='foreign_gross_x', ascending=False).head(20)

## Making Merged DataFrame into CSV

In [ ]:
cleancm.to_csv('/Users/jamiecarnevale/Documents/Flatiron/Projects/Phase 1/Movie_Production_Recommendations/Jamie/data/CSV_DB_Merge_V2.csv')

# Further Cleaning Merged DataFrame
- This will help me explore the data more efficiently

In [ ]:
df = cleancm[['title_x', 'genres', 'original_language', 'production_budget', 'worldwide_gross', 
                     'averagerating', 'release_date_x', 'domestic_gross_x', 'foreign_gross_x',
                      'popularity', 'vote_average', 'runtime_minutes',
                     'vote_count', 'release_year', 'studio', 'numvotes',
                        ]]

In [ ]:
df = df.rename(columns={'title_x':'Title', 'genres': 'Genres', 'original_language':'OL', 
                   'production_budget': 'PBudget', 'worldwide_gross':'WWGross', 
                     'averagerating':'AvRating', 'release_date_x':'RDate', 
                   'domestic_gross_x':'DGrossX', 'foreign_gross_x':'FGrossX',
                      'popularity':'Pop', 'vote_average':'VoteAv', 'runtime_minutes':'RuntimeMin',
                     'vote_count':'VoteCount', 'release_year':'RelYear', 'studio':'Studio', 'numvotes':'NumVotes'})

In [ ]:
df['PBudget'] = df['PBudget'].str.replace('$','')
df['PBudget'] = df['PBudget'].str.replace(',','')
df['PBudget'] = df['PBudget'].astype(int)

In [ ]:
df['Profit'] =  df['WWGross'] - df['PBudget']

In [ ]:
df = df[['Title', 'Genres', 'OL', 
                   'PBudget', 'WWGross', 'Profit',
                     'AvRating', 'RDate', 
                   'DGrossX', 'FGrossX',
                      'Pop', 'VoteAv', 'RuntimeMin',
                     'VoteCount', 'RelYear', 'Studio', 'NumVotes']]

In [ ]:
df['IsProfitable'] = df['Profit'] > 0

In [ ]:
df.sort_values(by='Profit', ascending=False).tail(10)

In [ ]:
df.info()

In [ ]:
df

In [ ]:
dfexplode = df.copy()
dfexplode['Genres'] = df['Genres'].str.split(',')
dfexplode = dfexplode.explode('Genres')
dfexplode

In [ ]:
dfexplode.head()

# Exploring Clean Data Visually

In [ ]:
#list to change variables around: 
# 'Profit','Pop','Genres','WWGross','AvRating','DGrossX','FGrossX','RuntimeMin','Studio','PBudget','IsProfitable'
dfexplodeexplore = dfexplode[['Profit','Pop','Genres','AvRating','WWGross', 'RelYear',
                                'RuntimeMin','IsProfitable', 'PBudget']]
dfexplodeexplore

In [ ]:
sns.pairplot(dfexplodeexplore, hue='IsProfitable', plot_kws={'alpha':0.3}, palette={True:'lightblue',False:'darkred'})

## Just looking at negative profit

In [ ]:
negprofit = dfexplode[dfexplode['Profit'] < 0]
negprofit.head()

In [ ]:
negprofitexplore = negprofit[['Profit','Pop','WWGross','AvRating', 'Genres',
                                'DGrossX','FGrossX','RuntimeMin','Studio','PBudget']]

In [ ]:
sns.pairplot(negprofitexplore)

## Looking at Genres

In [ ]:
(dfexplode[df['Genres'].str.contains('Thriller')]).sort_values(by='Profit').tail(30)

In [ ]:
x = dfexplodeexplore['Genres'].value_counts()
y = x[x > 20].index.tolist()
dfexplodeexploregenres = dfexplodeexplore[dfexplodeexplore['Genres'].isin(y)]
dfexplodeexploregenres.head(1)

In [ ]:
negprofitexploregenres = dfexplodeexploregenres[dfexplodeexploregenres['Profit'] < 0]
negprofitexploregenres.shape

In [ ]:
print(dfexplodeexplore['Genres'].value_counts())

In [ ]:
print(dfexplodeexploregenres['Genres'].value_counts())
sns.catplot(data=dfexplodeexploregenres, x='Genres', y = 'Profit', kind='box', height=5, aspect=2)
plt.xticks(rotation=90)
plt.show()

In [ ]:
columns = ['Profit','Pop','WWGross','AvRating', 'RuntimeMin']
print(dfexplodeexploregenres['Genres'].value_counts())
for col in columns:
    sns.catplot(data=dfexplodeexploregenres, x='Genres', y = col, kind='box', height=5, aspect=2,)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
dfexplodeexplore.shape

In [ ]:
columns = ['Profit','Pop','WWGross','AvRating', 'RuntimeMin','Studio']
print(negprofitexploregenres['Genres'].value_counts())
for col in columns:
    sns.catplot(data=negprofitexploregenres, x='Genres', y = col, kind='strip', height=5, aspect=2)
    plt.xticks(rotation=90)
    plt.show()

## Lets make these graphs a bit more pretty

In [ ]:
pbg = dfexplodeexploregenres.groupby('Genres')['Profit'].median().sort_values(ascending=False)
dfexplodeexploregenres['Genres'] = pd.Categorical(dfexplodeexploregenres['Genres'], categories=pbg.index, ordered=True)
pbg

In [ ]:
dfexplodeexploregenres.groupby('Genres')['Profit'].mean().sort_values(ascending=False)


In [ ]:
pbg2 = negprofitexploregenres.groupby('Genres')['Profit'].median().sort_values()
negprofitexploregenres['Genres'] = pd.Categorical(negprofitexploregenres['Genres'], categories=pbg2.index, ordered=True)
pbg2

In [ ]:
sns.set_theme(style='whitegrid')

In [ ]:
print(dfexplodeexploregenres['Genres'].value_counts())
sns.catplot(data=dfexplodeexploregenres, x='Genres', y = 'Profit', kind='box', height=5, aspect=2, palette='viridis')
plt.xticks(rotation=90)
plt.title('Best Performing Genres by Profit')
plt.show()

In [ ]:
print(negprofitexploregenres['Genres'].value_counts())
sns.catplot(data=negprofitexploregenres, x='Genres', y = 'Profit', kind='swarm', height=5, aspect=2)
plt.title('Unprofitable Genres')
plt.xticks(rotation=90)
plt.show()

# Creating a Graph that tests the Proportionality between Profitable Films and Unprofitable Films by Genre

In [ ]:
#percent of IsProfitable False / IsProfitable.valuecount

In [ ]:
dfexplodeexploregenres.groupby('Genres')['IsProfitable'].count()

In [ ]:
x = dfexplodeexplore['Genres'].value_counts()

In [ ]:

x = dfexplodeexploregenres.groupby('Genres')['IsProfitable'].count().sort_values(ascending=False)
y = dfexplodeexploregenres.groupby('Genres')['IsProfitable'].sum().sort_values(ascending=False)
z = (x-y)/x
percentprofit = z.reset_index()
percentprofit.columns = ['Genre', 'PercentNotProfitable']
percentprofit['PercentNotProfitable'] = percentprofit['PercentNotProfitable'] * 100
percentprofit = percentprofit.sort_values(by='PercentNotProfitable', ascending=False)
percentprofit

In [ ]:
fig, ax =plt.subplots(figsize=(16, 10))
sns.barplot(data=percentprofit, x='Genre', y='PercentNotProfitable', order=percentprofit['Genre'], palette='viridis',ax=ax)
print(dfexplodeexploregenres.groupby('Genres')['IsProfitable'].count())
ax.set_title('Percent of Financially Draining Films by Genre')
plt.xticks(rotation=90)
plt.show()

In [ ]:
dfexplodeexploregenres

# MORE DATA EXPLORATION, & DROP/RENAME/RE-ORDER COLUMNS FOR FOREIGN FOCUS

In [ ]:
Lou_Merged_Data = pd.read_csv('/Users/admin/Documents/Flatiron/Movie_Production_Recommendations/Lou/data/CSV_DB_Merge_V2.csv')

In [ ]:
Lou_Merged_Data['production_budget'] = Lou_Merged_Data['production_budget'].str.replace(',', '')
Lou_Merged_Data['production_budget'] = Lou_Merged_Data['production_budget'].str.replace('$', '')
Lou_Merged_Data['production_budget'] = Lou_Merged_Data['production_budget'].astype(float)

In [ ]:
Lou_Merged_Data.drop(['Unnamed: 0', 'foreign_gross_y', 'release_date_y', 'domestic_gross_y', 'release_date_y'], axis=1, inplace=True)

In [ ]:
Lou_Merged_Data.columns

In [ ]:
Lou_Merged_Data['domestic_gross_x'] = Lou_Merged_Data['domestic_gross_x'].astype(float)
Lou_Merged_Data['foreign_gross_x'] = Lou_Merged_Data['foreign_gross_x'].astype(float)

In [ ]:
Lou_Merged_Data.set_index('id_y')

In [ ]:
Lou_Merged_Data = Lou_Merged_Data.rename(columns={'id_y': 'id', 'title_x': 'title', 'numvotes': 'votes','release_date_x': 'release_date', 
                        'domestic_gross_x': 'domestic_gross', 'foreign_gross_x': 'foreign_gross', 
                        'averagerating': 'average_rating'})

In [ ]:
Lou_Merged_Data['domestic_gross'].isna().any()

In [ ]:
cols = Lou_Merged_Data.columns.tolist()
cols

In [ ]:
cols = ['id', 'title', 'genres', 'release_year', 'release_date', 'studio', 'vote_average', 'popularity','domestic_gross', 'foreign_gross', 'production_budget', 'runtime_minutes', 'average_rating', 'votes', 'worldwide_gross','original_language']  


In [ ]:
Lou_Merged_Data = Lou_Merged_Data[cols]

# EXPLORING FOREIGN AND DOMESTIC GROSS, CREATING FOREIGN- AND DOMESTIC-DOMINATED DATAFRAMES

In [ ]:
Lou_Merged_Data['foreign_gross'].isna().sum()

In [ ]:
Lou_Merged_Data['foreign_gross'].sample(100)

In [ ]:
test = Lou_Merged_Data[Lou_Merged_Data['foreign_gross'] > Lou_Merged_Data['domestic_gross']]

In [ ]:
foreign_gt_domestic = test

In [ ]:
net = foreign_gt_domestic['worldwide_gross'] - foreign_gt_domestic['production_budget']

foreign_gt_domestic['net'] = net

In [ ]:
foreign_gt_domestic['net'].mean()

In [ ]:
foreign_gt_domestic['net'].median()

In [ ]:
foreign_gt_domestic['net'].describe()

In [ ]:
net = Lou_Merged_Data['worldwide_gross'] - Lou_Merged_Data['production_budget']

Lou_Merged_Data['net'] = net

In [ ]:
Lou_Merged_Data['net'].mean()

In [ ]:
foreign_gt_domestic['net'].mean() - Lou_Merged_Data['net'].mean()

In [ ]:
domestic_gt_foreign =  Lou_Merged_Data[Lou_Merged_Data.domestic_gross.gt(Lou_Merged_Data.foreign_gross)]

In [ ]:
domestic_gt_foreign['net'].describe()

In [ ]:
foreign_gt_domestic.describe()

In [ ]:
foreign_gt_domestic['net'].sort_values(ascending=False).sample(50)

In [ ]:
Lou_Merged_Data['worldwide_gross'].describe()

In [ ]:
Lou_Merged_Data['net'].sort_values(ascending=False).head(100)

In [ ]:
domestic_gt_foreign =  Lou_Merged_Data[Lou_Merged_Data.domestic_gross.gt(Lou_Merged_Data.foreign_gross)]

In [ ]:
domestic_gt_foreign['net'].mean()

In [ ]:
domestic_gt_foreign['net'].describe()

In [ ]:
big_profit_difference = foreign_gt_domestic['net'].mean() - domestic_gt_foreign['net'].mean()
big_profit_difference

In [ ]:
domestic_gt_foreign['net'].describe()

In [ ]:
foreign_vs_domestic = Lou_Merged_Data.foreign_gross.gt(Lou_Merged_Data.domestic_gross)

Lou_Merged_Data['foreign_vs_domestic'] = foreign_vs_domestic 

Lou_Merged_Data['foreign_vs_domestic']

# PLOTTING FOREIGN GROSS-DOMINATED VS DOMESTIC-GROSS DOMINATED

In [ ]:
sns.pairplot(Lou_Merged_Data, corner = True)
plt.show()

In [ ]:
sns.pairplot(foreign_gt_domestic, corner=True)

In [ ]:
sns.PairGrid(Lou_Merged_Data, hue='foreign_gross', palette='hls').map(sns.scatterplot)
plt.show()

In [ ]:

sns.lmplot(x="domestic_gross", y="net", line_kws={"color": "C1"}, data=Lou_Merged_Data, x_estimator=np.mean)
sns.lmplot(x="foreign_gross", y="net", line_kws={"color": "C1"}, data=Lou_Merged_Data, x_estimator=np.mean)


In [ ]:
fig, ax = plt.subplots(figsize=(12,7))
sns.scatterplot(x='worldwide_gross', y='net', hue='foreign_vs_domestic', data=Lou_Merged_Data)
plt.show()
ax.set_xlabel = 'World Gross'
ax.set_ylabel = 'Profit'
ax.set_title = 'Profits by Foreign- and Domestic-Dominated Gross'


In [ ]:
g = sns.catplot(
    data=Lou_Merged_Data, kind="bar",
    x='foreign_vs_domestic', y="net",
    palette="dark", alpha=.6, height=6
).set(title='Profits by Foreign-Dominated Sales')
g.despine(left=True)
g.set_axis_labels("", "Profits (in Billions)")


In [ ]:
foreign_gt_domestic['genre_list'] = foreign_gt_domestic.genres.str.split(",")

foreign_gt_domestic

In [ ]:
foreign_gt_domestic['genre_list'].dropna(inplace=True)

In [ ]:
exploded_genres = foreign_gt_domestic.explode('genre_list')
exploded_genres 

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))

sns.boxplot(x='genre_list', y='net', data=exploded_genres)
plt.xticks(rotation=90)
plt.show()

In [ ]:
top_genres = exploded_genres.loc[(exploded_genres['genre_list'] == 'Action') | 
                                 (exploded_genres['genre_list'] == 'Adventure')|
                                 (exploded_genres['genre_list'] == 'Animation')|
                                 (exploded_genres['genre_list'] == 'Sci-Fi')|
                                 (exploded_genres['genre_list'] == 'Comedy')|
                                 (exploded_genres['genre_list'] == 'Family')
                                ]


In [ ]:
sns.barplot(x='genre_list', y = 'net', data=top_genres)
plt.show()

In [ ]:
genre_net = pd.DataFrame().assign(Genres=exploded_genres['genre_list'], Net=exploded_genres['net'])
genre_net

# LOOKING AT TOP GENRES BY PARTICULAR GENRE COMBINATIONS

In [ ]:
exploded_families = exploded_genres.loc[exploded_genres['genre_list'] == 'Family']

In [ ]:
exploded_families['genres'].value_counts()

In [ ]:
net_exploded_families = exploded_families['genres'].groupby(exploded_families['net']).sum().tail(5)
net_exploded_families

In [ ]:
exploded_action = exploded_genres.loc[exploded_genres['genre_list'] == 'Action']

In [ ]:
net_exploded_action = exploded_action['genres'].groupby(exploded_action['net']).sum().tail(5)
net_exploded_action

In [ ]:
test3 = Lou_Merged_Data.loc[Lou_Merged_Data['genres'] == 'Action,Adventure,Sci-Fi']
test3['net'].sort_values(ascending=False)

In [ ]:
exploded_adventure = exploded_genres.loc[exploded_genres['genre_list'] == 'Adventure']

In [ ]:
net_exploded_adventure = exploded_adventure['genres'].groupby(exploded_action['net']).sum().tail(5)
net_exploded_adventure

In [ ]:
exploded_animation = exploded_genres.loc[exploded_genres['genre_list'] == 'Animation']
exploded_sci_fi = exploded_genres.loc[exploded_genres['genre_list'] == 'Sci-Fi']
exploded_comedy = exploded_genres.loc[exploded_genres['genre_list'] == 'Comedy']
net_exploded_animation = exploded_animation['genres'].groupby(exploded_action['net']).sum().tail(5)
net_exploded_sci_fi = exploded_sci_fi['genres'].groupby(exploded_sci_fi['net']).sum().tail(5)
net_exploded_comedy = exploded_sci_fi['genres'].groupby(exploded_comedy['net']).sum().tail(5)

In [ ]:
net_exploded_animation

In [ ]:
net_exploded_sci_fi

In [ ]:
net_exploded_comedy

In [ ]:
exploded_genres['genre_list'].value_counts()

In [ ]:
AAA  = foreign_gt_domestic.loc[foreign_gt_domestic['genres'] == 'Action,Adventure,Animation']

In [ ]:
AAS  = foreign_gt_domestic.loc[foreign_gt_domestic['genres'] == 'Action,Adventure,Sci-Fi']
AAS

In [ ]:
top_genre_combos = foreign_gt_domestic.copy()

In [ ]:
top_genre_combos = top_genre_combos.loc[(top_genre_combos['genres'] == 'Action,Adventure,Sci-Fi') | 
                     (top_genre_combos['genres'] == 'Action,Comedy,Sci-Fi') | 
                     (top_genre_combos['genres'] == 'Action,Adventure,Animation') | 
                     (top_genre_combos['genres'] == 'Adventure,Family,Fantasy')]
top_genre_combos['genres'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
sns.barplot(x='genres', y='net', data=top_genre_combos)
plt.show()